#

# LP reader

This Notebook develops a .lp reader.

In [1]:
using SparseArrays
using LinearAlgebra
using DataStructures  # For OrderedDict if needed

push!(LOAD_PATH, realpath("../src"))
using LpProblem
using LpReadLP
using LpReadMPS

<details>
    <summary><h3> LPProblem Struct </h3></summary>

```juila
struct LPProblem
    is_minimize::Bool                     # True if the objective is to minimize
    c::Vector{Float64}                    # Objective function coefficients
    A::SparseMatrixCSC{Float64, Int64}    # Constraint matrix
    b::Vector{Float64}                    # Right-hand side of constraints
    constraint_types::Vector{Char}        # Constraint types ('L', 'G', 'E')
    l::Vector{Float64}                    # Lower bounds for variables
    u::Vector{Float64}                    # Upper bounds for variables
    vars::Vector{String}                  # Variable names
    variable_types::Vector{Symbol}        # Variable types
end
```

</details>

In [ ]:
lp_filepath = "../check/problems/lp_files/1451.lp"
 
lp = read_lp(lp_filepath)

# Accessing different components
println("Objective is to minimize: ", lp.is_minimize)
println("Objective coefficients: ", lp.c)
println("Constraint matrix A: ", lp.A)
println("Right-hand side vector b: ", lp.b)
println("Constraint types: ", lp.constraint_types)
println("Lower bounds: ", lp.l)
println("Upper bounds: ", lp.u)
println("Variables: ", lp.vars)
println("Variable types: ", lp.variable_types)


In [ ]:
println(read_file_to_string(lp_filepath))

In [ ]:
lp_filepath = "../check/problems/lp_files/juLinear_ex1.lp"
 
lp = read_lp(lp_filepath)

# Accessing different components
println("Objective is to minimize: ", lp.is_minimize)
println("Objective coefficients: ", lp.c)
println("Constraint matrix A: ", lp.A)
println("Right-hand side vector b: ", lp.b)
println("Constraint types: ", lp.constraint_types)
println("Lower bounds: ", lp.l)
println("Upper bounds: ", lp.u)
println("Variables: ", lp.vars)
println("Variable types: ", lp.variable_types)

In [ ]:
lp_problem

## Writing LP Files

<details>
    <summary>Write Lp Function</summary>

```julia
function write_lp(filename::String, problem::LPProblem; tolerance::Float64=1e-10)
    # Validate input dimensions
    n_vars = length(problem.vars)
    n_constraints = size(problem.A, 1)
    
    @assert length(problem.c) == n_vars "Objective coefficient vector length mismatch"
    @assert size(problem.A, 2) == n_vars "Constraint matrix column count mismatch"
    @assert length(problem.b) == n_constraints "RHS vector length mismatch"
    @assert length(problem.constraint_types) == n_constraints "Constraint types length mismatch"
    @assert length(problem.l) == n_vars "Lower bounds vector length mismatch"
    @assert length(problem.u) == n_vars "Upper bounds vector length mismatch"
    @assert length(problem.variable_types) == n_vars "Variable types length mismatch"
    
    # Validate constraint types
    valid_types = ['L', 'G', 'E']
    @assert all(t ∈ valid_types for t in problem.constraint_types) "Invalid constraint type found"
    
    function format_term(coeff::Float64, var::String, is_first::Bool)
        abs_coeff = abs(coeff)
        if abs_coeff < tolerance
            return ""
        elseif abs(abs_coeff - 1.0) < tolerance
            prefix = coeff < 0 ? "- " : (is_first ? "" : "+ ")
            return "$(prefix)$var"
        else
            prefix = coeff < 0 ? "-" : (is_first ? "" : "+")
            return "$(prefix) $abs_coeff $var"
        end
    end

    open(filename, "w") do io
        # Write Objective
        println(io, problem.is_minimize ? "Minimize" : "Maximize")
        print(io, " obj: ")
        
        # Write objective function
        terms = String[]
        first_term = true
        for (i, coeff) in enumerate(problem.c)
            term = format_term(coeff, problem.vars[i], first_term)
            if !isempty(term)
                push!(terms, term)
                first_term = false
            end
        end
        println(io, isempty(terms) ? "0" : join(terms, " "))

        # Write Constraints
        println(io, "Subject To")
        for i in 1:n_constraints
            terms = String[]
            first_term = true
            for (j, coeff) in zip(findnz(problem.A[i, :])...)
                term = format_term(coeff, problem.vars[j], first_term)
                if !isempty(term)
                    push!(terms, term)
                    first_term = false
                end
            end
            
            relation = Dict('L' => "<=", 'G' => ">=", 'E' => "=")[problem.constraint_types[i]]
            println(io, " c$i: ", isempty(terms) ? "0" : join(terms, " "), " $relation $(problem.b[i])")
        end

        # Write Bounds
        println(io, "Bounds")
        for i in 1:n_vars
            var = problem.vars[i]
            lower = problem.l[i]
            upper = problem.u[i]
            
            if abs(lower - upper) < tolerance
                println(io, " $var = $lower")
            elseif lower > -Inf && upper < Inf
                println(io, " $lower <= $var <= $upper")
            elseif lower > -Inf
                println(io, " $var >= $lower")
            elseif upper < Inf
                println(io, " $var <= $upper")
            else
                println(io, " $var free")
            end
        end

        # Write Binary and Integer variables
        for var_type in [:Binary, :Integer]
            vars_of_type = [problem.vars[i] for i in 1:n_vars if problem.variable_types[i] == var_type]
            if !isempty(vars_of_type)
                println(io, var_type == :Binary ? "Binary" : "General")
                for var in vars_of_type
                    println(io, " $var")
                end
            end
        end

        println(io, "End")
    end
end
```

</details>

In [4]:
# lp_filepath = "../check/problems/mps_files/test.mps"
# lp = read_mps_from_file(lp_filepath)
# write_lp("../check/problems/lp_files/test.lp", lp)

In [7]:
# lp_filepath = "../check/problems/mps_files/ex_9-7.mps"
# lp = read_mps_from_file(lp_filepath)
# write_lp("../check/problems/lp_files/ex_9-7.lp", lp)

In [2]:
lp_filepath = "../check/problems/mps_files/ex4-3.mps"
lp = read_mps(lp_filepath)
write_lp("../check/problems/lp_files/ex4-3.lp",lp)

In [10]:
function Base.:(==)(lp1::LPProblem, lp2::LPProblem)
    return lp1.is_minimize == lp2.is_minimize &&
           lp1.c == lp2.c &&
           lp1.A == lp2.A &&
           lp1.b == lp2.b &&
           lp1.constraint_types == lp2.constraint_types &&
           lp1.l == lp2.l &&
           lp1.u == lp2.u &&
           lp1.vars == lp2.vars &&
           lp1.variable_types == lp2.variable_types
end


In [12]:
println(lp)
println(read_lp("../check/problems/lp_files/ex4-3.lp"))
println(lp == read_lp("../check/problems/lp_files/ex4-3.lp")) 

LPProblem(true, [0.0, 0.0, 0.0, 0.0, 0.0], sparse([1, 2, 1, 2, 1, 2, 1, 2, 1, 2], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5], [4.0, -3.0, -3.0, 2.0, -2.0, -1.0, 1.0, 2.0, 2.0, 3.0], 2, 5), [13.0, -9.0], ['L', 'L'], [0.0, 0.0, 1.0, 2.0, 0.0], [Inf, 3.0, 5.0, 4.0, Inf], ["x1", "x2", "x3", "y4", "y5"], [:Continuous, :Continuous, :Continuous, :Continuous, :Continuous])
LPProblem(true, [0.0, 0.0, 0.0, 0.0, 0.0], sparse([1, 2, 1, 2, 1, 2, 1, 2, 1, 2], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5], [4.0, -3.0, -3.0, 2.0, -2.0, -1.0, 1.0, 2.0, 2.0, 3.0], 2, 5), [13.0, -9.0], ['L', 'L'], [0.0, 0.0, 1.0, 2.0, 0.0], [Inf, 3.0, 5.0, 4.0, Inf], ["x1", "x2", "x3", "y4", "y5"], [:Continuous, :Continuous, :Continuous, :Continuous, :Continuous])
true
